In [ ]:
'''
#!pip install transformers
#!pip install torch
%pip install torch==1.13.1 transformers==4.26.1 --quiet



from pyspark.sql import SparkSession
from pyspark.sql.functions import col, monotonically_increasing_id
from transformers import pipeline
import torch


DELTA_PATH = "/mnt/nyt/archive_yearly"  # Your previously ingested data path
MODEL = "assemblyai/distilbert-base-uncased-sst2"  # Lightweight, accurate model
BATCH_SIZE = 64                            # Adjust for memory (Databricks CE = 15GB RAM)

# Load data
spark = SparkSession.builder.getOrCreate()
df = spark.read.format("delta").load(DELTA_PATH)
df = df.filter(col("headline").isNotNull())  # Remove nulls

# Add a unique ID to every row for safe join later
df_with_id = df.withColumn("row_id", monotonically_increasing_id())


# 4. COLLECT HEADLINES LOCALLY FOR SENTIMENT ANALYSIS
rows = df_with_id.select("row_id", "headline").collect()
headlines = [(row["row_id"], row["headline"]) for row in rows]


# 5. LOAD SENTIMENT ANALYSIS PIPELINE
# NOTE: Avoid `device_map="cuda"` in CE (no GPU support)
sentiment_pipeline = pipeline("sentiment-analysis", model=MODEL, tokenizer=MODEL, truncation=True)

# Run batch sentiment analysis
results = []
for row_id, headline in headlines:
    try:
        pred = sentiment_pipeline(headline)[0]
        label = "positive" if pred["label"] == "LABEL_1" else "negative"
        score = float(pred["score"])
        results.append((row_id, headline, label, score))
    except Exception as e:
        results.append((row_id, headline, "error", 0.0))  # fallback on error
        print(f"Error processing row_id {row_id}: {e}")

# --------------------------------------
# 6. CREATE SPARK DATAFRAME WITH SENTIMENT
sentiment_schema = ["row_id", "headline", "sentiment_label", "sentiment_score"]
sentiment_df = spark.createDataFrame(results, sentiment_schema)
sentiment_df = sentiment_df.withColumnRenamed("headline", "headline_sentiment")


# --------------------------------------
# 7. JOIN BACK TO ORIGINAL DATAFRAME
augmented_df = df_with_id.join(sentiment_df, on="row_id", how="left")

# --------------------------------------
# 8. DISPLAY RESULTS
augmented_df.select(
    "headline", "sentiment_label", "sentiment_score", "pub_date"
).orderBy("sentiment_score", ascending=False).show(20, truncate=False)

# --------------------------------------
# 9. (OPTIONAL) SAVE TO DELTA TABLE
OUTPUT_PATH = "/mnt/nyt/sentiment_augmented"
augmented_df.write.format("delta").mode("overwrite").save(OUTPUT_PATH)

# 1. Imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_timestamp, window,
    collect_list, count, monotonically_increasing_id
)
from transformers import pipeline

#
MODEL_NAME = "assemblyai/distilbert-base-uncased-sst2"
BATCH_SIZE  = 64               # tune if OOM / too slow

spark = SparkSession.builder.getOrCreate()

# 2. Load & prepare the NYT archive
articles = (
    spark.table("nyt_archive")                               # <- your Delta table
         .filter(col("headline").isNotNull())                
         .withColumn("timestamp", to_timestamp(col("pub_date")))
         .select("timestamp", "headline", "topic")
)

# Group articles
grouped = (
    articles
      .groupBy(
          window("timestamp", "24 hours").alias("time_window"),
          col("topic")
      )
      .agg(
          collect_list("headline").alias("headlines"),        # keep them as a list
          count("*").alias("article_count")
      )
      .select(
          col("time_window.start").alias("window_start"),
          col("time_window.end").alias("window_end"),
          "topic",
          "headlines",
          "article_count"
      )
      .withColumn("group_id", monotonically_increasing_id())  # safe join key
)

# Run Sentiment Analysis
groups_local = grouped.select("group_id", "headlines").collect()

sentiment_pipe = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    truncation=True,
    batch_size=BATCH_SIZE
)

results = []
for row in groups_local:
    gid        = row["group_id"]
    headlines  = row["headlines"]

    # Model inference (batched internally by HF pipeline)
    preds = sentiment_pipe(headlines)

    # Convert HF labels → human-readable
    labels = ["positive" if p["label"] == "LABEL_1" else "negative" for p in preds]

    pos_cnt = labels.count("positive")
    neg_cnt = labels.count("negative")
    maj_sent = "positive" if pos_cnt >= neg_cnt else "negative"  # tie → positive

    results.append((gid, pos_cnt, neg_cnt, maj_sent))

# Build a tiny DataFrame to join back
schema = ["group_id", "positive_count", "negative_count", "majority_sentiment"]
sentiment_df = spark.createDataFrame(results, schema)

# Final dataframe
final_df = (
    grouped
      .join(sentiment_df, on="group_id", how="left")
      .select(
          "window_start", "window_end", "topic",
          "article_count", "positive_count", "negative_count",
          "majority_sentiment"
      )
      .orderBy("window_start", "topic")
)


final_df.show(truncate=False)

# save as Delta 
OUTPUT_PATH = "/mnt/nyt/sentiment_daily_topic"
final_df.write.format("delta").mode("overwrite").save(OUTPUT_PATH)
'''

In [ ]:
# ────────────────────────────────────────────────────────────────────────────────
# 0. (Re-)install lightweight HF model – one-time per cluster
# ────────────────────────────────────────────────────────────────────────────────
%pip uninstall -y torch transformers
%pip install --quiet torch==1.13.1 transformers==4.26.1

# ────────────────────────────────────────────────────────────────────────────────
# 1. Imports & helpers
# ────────────────────────────────────────────────────────────────────────────────
import re
import torch
from transformers import pipeline

from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, to_timestamp, window, collect_list, count,
    monotonically_increasing_id, first
)

def sanitize(name: str) -> str:
    """Turn an arbitrary topic into a safe column name."""
    return re.sub(r"[^A-Za-z0-9]+", "_", name.strip()).lower()

MODEL_NAME = "assemblyai/distilbert-base-uncased-sst2"
BATCH_SIZE  = 64                       # adjust for memory

spark = SparkSession.builder.getOrCreate()

# ────────────────────────────────────────────────────────────────────────────────
# 2. Pull the headline corpus
# ────────────────────────────────────────────────────────────────────────────────
articles = (
    spark.table("nyt_archive")
         .filter(col("headline").isNotNull())
         .withColumn("timestamp", to_timestamp(col("pub_date")))
         .select("timestamp", "headline", "topic")
)

# ────────────────────────────────────────────────────────────────────────────────
# 3. 24-hour tumbling window × topic   (list of headlines per group)
# ────────────────────────────────────────────────────────────────────────────────
groups = (
    articles
      .groupBy(window("timestamp", "24 hours").alias("tw"), col("topic"))
      .agg(
          collect_list("headline").alias("headlines"),
          count("*").alias("article_count")
      )
      .select(
          col("tw.start").alias("window_start"),
          col("tw.end").alias("window_end"),
          "topic",
          "headlines",
          "article_count"
      )
      .withColumn("group_id", monotonically_increasing_id())
)

# ────────────────────────────────────────────────────────────────────────────────
# 4. Driver-side sentiment inference (HF pipeline batches internally)
# ────────────────────────────────────────────────────────────────────────────────
groups_local = groups.select("group_id", "headlines").collect()

sent_pipe = pipeline(
    "sentiment-analysis",
    model=MODEL_NAME,
    tokenizer=MODEL_NAME,
    batch_size=BATCH_SIZE,
    truncation=True
)

sent_results = []
for row in groups_local:
    gid, hl_list = row["group_id"], row["headlines"]
    preds   = sent_pipe(hl_list)
    labels  = ["positive" if p["label"] == "LABEL_1" else "negative" for p in preds]
    pos, neg = labels.count("positive"), labels.count("negative")
    majority = "positive" if pos >= neg else "negative"           # tie → positive
    sent_results.append((gid, pos, neg, majority))

sent_df = spark.createDataFrame(
    sent_results,
    ["group_id", "positive_count", "negative_count", "majority_sentiment"]
)

# ────────────────────────────────────────────────────────────────────────────────
# 5. Join back – one row per window × topic
# ────────────────────────────────────────────────────────────────────────────────
daily_topic_df = (
    groups
      .join(sent_df, on="group_id", how="left")
      .select(
          "window_start", "window_end", "topic",
          "article_count", "positive_count", "negative_count",
          "majority_sentiment"
      )
)

# ────────────────────────────────────────────────────────────────────────────────
# 6. Pivot so **each topic becomes a column**  (sentiment & counts)
# ────────────────────────────────────────────────────────────────────────────────
from functools import reduce

# --- Pivot majority sentiment --------------------------------------------------
sent_pivot = (
    daily_topic_df
      .groupBy("window_start", "window_end")
      .pivot("topic")
      .agg(first("majority_sentiment"))
)

# --- Pivot article counts  --------
counts_pivot = (
    daily_topic_df
      .groupBy("window_start", "window_end")
      .pivot("topic")
      .agg(first("article_count"))
)

# --- Rename columns to safe, prefixed versions ---------------------------------
sent_safe   = sent_pivot
counts_safe = counts_pivot
for c in sent_pivot.columns:
    if c not in ("window_start", "window_end"):
        clean = sanitize(c)
        sent_safe   = sent_safe  .withColumnRenamed(c, f"sent_{clean}")
        counts_safe = counts_safe.withColumnRenamed(c, f"count_{clean}")

# ────────────────────────────────────────────────────────────────────────────────
# 7. Combine sentiment + count matrices (optional)
# ────────────────────────────────────────────────────────────────────────────────
final_wide = (
    sent_safe
      .join(counts_safe, ["window_start", "window_end"], "inner")
      .orderBy("window_start")
)

# ────────────────────────────────────────────────────────────────────────────────
# 8. Inspect & persist
# ────────────────────────────────────────────────────────────────────────────────
display(final_wide)
# write into Delta (append mode), partitioned

DELTA_PATH  = "/mnt/nyt/news_sentiment"
(
final_wide.write \
        .format("delta") \
        .mode("overwrite") \
        .save(DELTA_PATH)
)
# 4) Register as a Hive table for easy querying
spark.sql(f"""
  CREATE TABLE IF NOT EXISTS news_sentiment
  USING DELTA
  LOCATION '{DELTA_PATH}'
""")


# Persist for downstream ML / dashboards
# OUTPUT_PATH = "/mnt/nyt/sentiment_daily_topic_wide"
# final_wide.write.format("delta").mode("overwrite").save(OUTPUT_PATH)


In [ ]:
# ────────────────────────────────────────────────────────────────
# Graphs & visualizations
# ────────────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# 0. CONFIG ──────────────────────────────────────────────────────
FILE = "/mnt/data/BDA_Sent_Analysis.xlsx"   # adjust path if needed
PLOT_STYLE = dict(color="royalblue", alpha=0.7)

# 1. LOAD & PREP ─────────────────────────────────────────────────
df = pd.read_excel(FILE)

sent_cols  = [c for c in df.columns if c.startswith("sent_")]
count_cols = [c for c in df.columns if c.startswith("count_")]

# helper → numeric: +1 / –1 / NaN
def lbl(x):
    x = str(x).lower()
    return 1 if x == "positive" else -1 if x == "negative" else np.nan

sent_num = df[sent_cols].applymap(lbl)

# daily pos / neg counts (NaN ignored) + ratio
df["pos_cnt"] = (sent_num == 1).sum(axis=1)
df["neg_cnt"] = (sent_num == -1).sum(axis=1)
df["pos_neg_ratio"] = df.apply(
    lambda r: r["pos_cnt"] / r["neg_cnt"] if r["neg_cnt"] > 0 else np.nan,
    axis=1
)

# topic-level aggregated counts
topic_pos = {c[5:]: (df[c].str.lower() == "positive").sum() for c in sent_cols}
topic_neg = {c[5:]: (df[c].str.lower() == "negative").sum() for c in sent_cols}
topic_vol = {c[6:]: df[c].fillna(0).sum() for c in count_cols}

topic_ratio = {
    k: (topic_pos[k] / topic_neg[k]) if topic_neg[k] > 0 else np.nan
    for k in topic_pos
}
ratio_ser = pd.Series(topic_ratio).dropna().sort_values()

# 2. SA-Fig 1 — Daily line of POS/NEG ratio ─────────────────────
dates = pd.to_datetime(df["window_start"])
plt.figure(figsize=(12, 4.5))
plt.plot(dates, df["pos_neg_ratio"], **PLOT_STYLE, lw=2)
plt.axhline(1, ls="--", color="grey", lw=0.8)
plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%b-%y"))
plt.xticks(rotation=45)
plt.ylabel("Positive / Negative ratio")
plt.title("Daily positive-to-negative sentiment ratio")
plt.tight_layout()
plt.savefig("SA_Fig1_ratio_line.png", dpi=300)
plt.close()

# 3. SA-Fig 2 — Topic bar of ratios ─────────────────────────────
plt.figure(figsize=(9, 12))
plt.barh(ratio_ser.index, ratio_ser.values, **PLOT_STYLE)
plt.axvline(1, color="black", lw=0.8)
plt.xlabel("Positive / Negative ratio (Jan–Apr 2025)")
plt.title("Topic-level sentiment ratio (NaN where never negative)")
plt.tight_layout()
plt.savefig("SA_Fig2_topic_bar.png", dpi=300)
plt.close()

# 4. SA-Fig 3 — Volume vs ratio scatter (circle bubbles) ───────
rows = [
    (t, topic_vol[t], topic_ratio[t])
    for t in topic_ratio.keys()
    if not np.isnan(topic_ratio[t])
]
scat = pd.DataFrame(rows, columns=["topic", "volume", "ratio"])

plt.figure(figsize=(9, 6))
plt.scatter(
    scat["volume"],
    scat["ratio"],
    s=scat["volume"] / 5 + 20,
    marker="o",
    **PLOT_STYLE
)
for _, r in scat.nlargest(12, "volume").iterrows():
    plt.text(r["volume"], r["ratio"] + 0.05, r["topic"], fontsize=8)
plt.axhline(1, ls="--", color="grey", lw=0.8)
plt.xlabel("Total articles (null→0)")
plt.ylabel("Positive / Negative ratio")
plt.title("Coverage volume vs sentiment ratio by topic (Jan–Apr 2025)")
plt.tight_layout()
plt.savefig("SA_Fig3_scatter.png", dpi=300)
plt.close()

print("✓ SA-Fig1_ratio_line.png\n✓ SA-Fig2_topic_bar.png\n✓ SA-Fig3_scatter.png")
